In [1]:
# Необходимые библиотеки
import requests
from bs4 import BeautifulSoup as bs
from pprint import pprint
import re
import pandas as pd

In [2]:
# Сохранение путей и параметров в переменные
url = 'https://hh.ru'
params = {'clusters': 'true',
          'ored_clusters': 'true',
          'enable_snippets': 'true',
          'salary': '',
          'text': 'data scientist',
          'page': 0}
headers = {'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/95.0.4638.69 Safari/537.36'}

In [3]:
vacancies_list = []
while True:
    response = requests.get(url + '/search/vacancy/', params=params, headers=headers)
    dom = bs(response.text, 'html.parser')
    vacancies = dom.find_all('div', {'class': 'vacancy-serp-item'})
    
    if response.ok and vacancies:
        for vacancy in vacancies:
            vacancies_data = {}
            _id = vacancy.find('a', {'class': 'bloko-link'})['href'].split('?')[0].split('/')[-1]            
            name = vacancy.find('a', {'class': 'bloko-link'}).text
            link = vacancy.find('a', {'class': 'bloko-link'})['href']
            company_name = vacancy.find('div', {'class': 'vacancy-serp-item__meta-info-company'}).text
            adress = vacancy.find('div', {'data-qa': 'vacancy-serp__vacancy-address'}).text
            wage = []
            try:
                for i in re.split(" – | ", vacancy.find('span', {'data-qa': 'vacancy-serp__vacancy-compensation'}).text):
                    i = i.split('\u202f')
                    wage.append(''.join(i))
                min_wage = int(wage[0])
                max_wage = int(wage[1])
                currency = wage[2]
            except:
                try:
                    if wage[0] == 'до':
                        min_wage = None
                        max_wage = int(wage[1])
                        currency = wage[2]
                    elif wage[0] == 'от':
                        min_wage = int(wage[1])
                        max_wage = None
                        currency = wage[2]
                except:
                    min_wage = None
                    max_wage = None
                    currency = None
            vacancies_data['_id'] = _id
            vacancies_data['name'] = name
            vacancies_data['link'] = link
            vacancies_data['company_name'] = company_name
            vacancies_data['adress'] = adress
            vacancies_data['min_wage'] = min_wage
            vacancies_data['max_wage'] = max_wage
            vacancies_data['currency'] = currency
            vacancies_list.append(vacancies_data)
        params['page'] += 1
    else:
        break

In [4]:
df = pd.DataFrame.from_dict(vacancies_list)
df.to_csv(f'hh_{params["text"]}.csv')

In [22]:
df

,_id,name,link,company_name,adress,min_wage,max_wage,currency
0,click,Senior Data Scientist,https://hhcdn.ru/click?b=253624&place=35,Сбер. Data Science,Сочи,NaN,NaN,None
1,46048820,Data Scientist (команда Поиска),https://hh.ru/vacancy/46048820?from=vacancy_se...,ООО HeadHunter::Analytics/Data Science,"Москва, Алексеевская",NaN,NaN,None
2,49232800,Python Developer (Data Science),https://hh.ru/vacancy/49232800?from=vacancy_se...,ООО IHS Global,Москва,NaN,NaN,None
3,47377173,Data Scientist,https://hh.ru/vacancy/47377173?from=vacancy_se...,Сбер для экспертов,Екатеринбург,NaN,NaN,None
4,49232780,Senior Data Scientist (Computer Vision) - Synt...,https://hh.ru/vacancy/49232780?from=vacancy_se...,ООО IHS Global,Москва,NaN,NaN,None
...,...,...,...,...,...,...,...,...
315,48530633,Quant Analyst,https://hh.ru/vacancy/48530633?from=vacancy_se...,VLG Capital,Москва,NaN,NaN,None
316,48576040,Product Marketing Manager (DataSpell),https://hh.ru/vacancy/48576040?from=vacancy_se...,JetBrains,Санкт-Петербург,150000.0,NaN,руб.
317,48516841,Market Analyst,https://hh.ru/vacancy/48516841?from=vacancy_se...,NielsenIQ,Москва,NaN,NaN,None
318,48576039,Product Marketing Manager (DataSpell),https://hh.ru/vacancy/48576039?from=vacancy_se...,JetBrains,Москва,150000.0,NaN,руб.


# Домашнее задание 3 урока

### 1 часть

In [75]:
from pymongo import MongoClient
from pymongo.errors import DuplicateKeyError as dke

In [76]:
client = MongoClient('127.0.0.1', 27017)
db = client['hhvacancies']
vacancies = db.vacancies

In [85]:
def add_documents(value):
    if type(value) == list:
        for el in value:
            try:
                vacancies.insert_one(el)
            except TypeError:
                print('Неверный тип данных')
            except dke:
                print(f"Вакансия с id {el['_id']} уже добавлена")  
    else:
        try:
            vacancies.insert_one(value)
        except TypeError:
            print('Неверный тип данных')
        except dke:
            print("Такая вакансия уже добавлена")

### 2 часть

In [119]:
def look_for_documents(wage):
    out = []

    for item in vacancies.find({'$or': [{'min_wage': {'$gt': 0}}, {'max_wage': {'$gt': 0}}]}): #нет смысла выводить то, где пустые значения в макс и мин ЗП
        if item["min_wage"] == None:
            if wage <= item['max_wage']:
                out.append(item)
        elif item["max_wage"] == None:
            if wage <= item["min_wage"]:
                out.append(item)
        else:
            if wage <= item["min_wage"]:
                out.append(item)
            elif wage <= item["max_wage"]:
                out.append(item)
    return out

In [ ]:
look_for_documents(200000)